In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import numpy as np
from glob import glob
from pathlib import Path
import matplotlib.pyplot as plt
import pandas as pd 
import seaborn as sns
import matplotlib
from math import pow, sqrt, ceil
from collections import OrderedDict

from sklearn.metrics import accuracy_score, cohen_kappa_score, confusion_matrix

In [3]:
from Analysis.annotations import Annotation
from Analysis.image import Image
from Analysis.expert import Expert, DatasetType, ProjectType

In [4]:
from utils import *

In [5]:
path = Path("Results/Mitosen/FirstRound")
files_first_round = glob(str(path)+"/Participants/*.txt")
files_second_round = glob(str(path).replace("First", "Second")+"/Participants/*.txt")
gt_path = str(path)+"/GT/ground truth.txt"


users_first_round = []
users_second_round = {}
ground_truth = Expert("GT", BBType.GroundTruth, DatasetType.MitoticFigure, ProjectType.Annotation)
ground_truth.add_file(gt_path)


for file_name in files_first_round:
    participant = "Expert_" + Path(file_name).stem.split("_")[-1]

    expert = Expert(participant, BBType.Detected, DatasetType.MitoticFigure, ProjectType.Annotation)
    expert.add_file(file_name)

    users_first_round.append(expert)
    
for file_name in files_second_round:
    participant = "Expert_" + Path(file_name).stem.split("_")[-1]

    expert = Expert(participant, BBType.Detected, DatasetType.MitoticFigure, ProjectType.ExpertAlgorithm)
    expert.add_file(file_name)

    users_second_round[expert.expert] = expert

In [6]:
fake_mitosis_path = str(path)+"/GT/fake_mitosis.txt"

fake_mitosis_path_0 = str(path)+"/GT/FakeMitoticFigures_G0.txt"
fake_mitosis_path_1 = str(path)+"/GT/FakeMitoticFigures_G1.txt"
fake_mitosis_path_2 = str(path)+"/GT/FakeMitoticFigures_G2.txt"

fake_mitosis = Expert("fake_mitosis", BBType.GroundTruth, DatasetType.MitoticFigure, ProjectType.Annotation)
fake_mitosis.add_file(fake_mitosis_path)

fake_mitosis_0= Expert("fake_mitosis_0", BBType.GroundTruth, DatasetType.MitoticFigure, ProjectType.Annotation)
fake_mitosis_1= Expert("fake_mitosis_1", BBType.GroundTruth, DatasetType.MitoticFigure, ProjectType.Annotation)
fake_mitosis_2= Expert("fake_mitosis_2", BBType.GroundTruth, DatasetType.MitoticFigure, ProjectType.Annotation)

fake_mitosis_0.add_file(fake_mitosis_path_0)
fake_mitosis_1.add_file(fake_mitosis_path_1)
fake_mitosis_2.add_file(fake_mitosis_path_2)

In [7]:
deleted_mitosis_path = str(path)+"/GT/deleted_mitosis.txt"

deleted_mitosis_path_0 = str(path)+"/GT/RemovedMitoticFigures_G0.txt"
deleted_mitosis_path_1 = str(path)+"/GT/RemovedMitoticFigures_G1.txt"
deleted_mitosis_path_2 = str(path)+"/GT/RemovedMitoticFigures_G2.txt"

deleted_mitosis = Expert("deleted_mitosis", BBType.GroundTruth, DatasetType.MitoticFigure, ProjectType.Annotation)
deleted_mitosis.add_file(deleted_mitosis_path)


deleted_mitosis_0 = Expert("deleted_mitosis_0", BBType.GroundTruth, DatasetType.MitoticFigure, ProjectType.Annotation)
deleted_mitosis_1 = Expert("deleted_mitosis_1", BBType.GroundTruth, DatasetType.MitoticFigure, ProjectType.Annotation)
deleted_mitosis_2 = Expert("deleted_mitosis_2", BBType.GroundTruth, DatasetType.MitoticFigure, ProjectType.Annotation)

deleted_mitosis_0.add_file(deleted_mitosis_path_0)
deleted_mitosis_1.add_file(deleted_mitosis_path_1)
deleted_mitosis_2.add_file(deleted_mitosis_path_2)

In [8]:
ground_truth

GT:  Annos: 712 Seconds: 0.0 Type: MitoticFigure Project: Annotation

In [9]:
print("First Round")
for user in users_first_round:
    print(str(user) + " mIoU: {0} Sens: {1} Spec: {2}".format(user.calc_MIoU(ground_truth), user.calc_sensitivity(ground_truth), user.calc_specificity(ground_truth, fake_mitosis)))
    
print("\nSecond Round")
for user in users_second_round.values():
    print(str(user) + " mIoU: {0} Sens: {1} Spec: {2}".format(user.calc_MIoU(ground_truth), user.calc_sensitivity(ground_truth), user.calc_specificity(ground_truth, fake_mitosis)))

First Round
Expert_11:  Annos: 637 Seconds: 242.05 Type: MitoticFigure Project: Annotation mIoU: 0.48224342864926656 Sens: 0.6530898876404494 Spec: 0.36531365313653136
Expert_12:  Annos: 334 Seconds: 478.55 Type: MitoticFigure Project: Annotation mIoU: 0.30746789834924293 Sens: 0.3595505617977528 Spec: 0.6320754716981132
Expert_13:  Annos: 351 Seconds: 206.0 Type: MitoticFigure Project: Annotation mIoU: 0.40034286483265796 Sens: 0.4410112359550562 Spec: 0.7757575757575758
Expert_3:  Annos: 654 Seconds: 137.5 Type: MitoticFigure Project: Annotation mIoU: 0.5352151549917714 Sens: 0.6825842696629213 Spec: 0.3684210526315789
Expert_4:  Annos: 694 Seconds: 590.75 Type: MitoticFigure Project: Annotation mIoU: 0.37187773032582927 Sens: 0.5997191011235955 Spec: 0.2917771883289125
Expert_5:  Annos: 884 Seconds: 420.65 Type: MitoticFigure Project: Annotation mIoU: 0.47205491965977137 Sens: 0.7556179775280899 Spec: 0.1761904761904762
Expert_6:  Annos: 673 Seconds: 363.45 Type: MitoticFigure Proje

### Found deleted

In [10]:
metrics = []
for user in users_first_round:
    metric_0 = user.calc_metrics(deleted_mitosis_0)[0]    
    metric_1 = user.calc_metrics(deleted_mitosis_1)[0]
    metric_2 = user.calc_metrics(deleted_mitosis_2)[0]
    
    metric_second_0 = None
    metric_second_1 = None
    metric_second_2 = None
    if user.expert in users_second_round:
        metric_second_0 = users_second_round[user.expert].calc_metrics(deleted_mitosis_0)[0]    
        metric_second_1 = users_second_round[user.expert].calc_metrics(deleted_mitosis_1)[0]
        metric_second_2 = users_second_round[user.expert].calc_metrics(deleted_mitosis_2)[0]
    
    row = [user.expert, int(metric_0['total positives']), int(metric_0['total TP']), int(metric_second_0['total TP']) if metric_second_0 is not None else 0,
           int(metric_1['total positives']), int(metric_1['total TP']), int(metric_second_1['total TP']) if metric_second_1 is not None else 0,
           int(metric_2['total positives']), int(metric_2['total TP']), int(metric_second_2['total TP']) if metric_second_2 is not None else 0,]
    
    
    metrics.append(row)
    
metrics.append(["Total"] + [sum([row[i] for row in metrics]) for i in range(1, 10)])

pd.DataFrame(metrics, columns=['Name', 'Group 0', "Annotation", "ExpertAlgorithm", 
                               'Group 1', "Annotation", "ExpertAlgorithm", 
                               'Group 2', "Annotation", "ExpertAlgorithm"])

,Name,Group 0,Annotation,ExpertAlgorithm,Group 1,Annotation,ExpertAlgorithm,Group 2,Annotation,ExpertAlgorithm
0,Expert_11,48,1,0,49,2,2,50,3,3
1,Expert_12,48,0,0,49,2,0,50,2,0
2,Expert_13,48,1,0,49,0,0,50,1,0
3,Expert_3,48,1,0,49,2,0,50,1,0
4,Expert_4,48,0,0,49,1,0,50,0,0
5,Expert_5,48,0,1,49,1,2,50,1,0
6,Expert_6,48,0,1,49,0,1,50,1,2
7,Expert_7,48,0,0,49,2,0,50,2,0
8,Expert_8,48,0,1,49,2,1,50,3,1
9,Expert_9,48,0,0,49,2,0,50,2,0


### Found fake mitosis 

In [11]:
metrics = []
for user in users_first_round:
    metric_0 = user.calc_metrics(fake_mitosis_0)[0]    
    metric_1 = user.calc_metrics(fake_mitosis_1)[0]
    metric_2 = user.calc_metrics(fake_mitosis_2)[0]
    
    metric_second_0 = None
    metric_second_1 = None
    metric_second_2 = None
    if user.expert in users_second_round:
        metric_second_0 = users_second_round[user.expert].calc_metrics(fake_mitosis_0)[0]    
        metric_second_1 = users_second_round[user.expert].calc_metrics(fake_mitosis_1)[0]
        metric_second_2 = users_second_round[user.expert].calc_metrics(fake_mitosis_2)[0]
    
    row = [user.expert, int(metric_0['total positives']), int(metric_0['total TP']), int(metric_second_0['total TP']) if metric_second_0 is not None else 0,
           int(metric_1['total positives']), int(metric_1['total TP']), int(metric_second_1['total TP']) if metric_second_1 is not None else 0,
           int(metric_2['total positives']), int(metric_2['total TP']), int(metric_second_2['total TP']) if metric_second_2 is not None else 0,]
    
    
    metrics.append(row)
    
metrics.append(["Total"] + [sum([row[i] for row in metrics]) for i in range(1, 10)])

pd.DataFrame(metrics, columns=['Name', 'Group 0', "Annotation", "ExpertAlgorithm", 
                               'Group 1', "Annotation", "ExpertAlgorithm", 
                               'Group 2', "Annotation", "ExpertAlgorithm"])

,Name,Group 0,Annotation,ExpertAlgorithm,Group 1,Annotation,ExpertAlgorithm,Group 2,Annotation,ExpertAlgorithm
0,Expert_11,48,0,0,49,2,0,50,2,2
1,Expert_12,48,0,0,49,0,0,50,2,0
2,Expert_13,48,0,0,49,0,0,50,0,0
3,Expert_3,48,1,0,49,1,0,50,2,0
4,Expert_4,48,0,0,49,0,0,50,1,0
5,Expert_5,48,1,1,49,0,0,50,1,1
6,Expert_6,48,1,1,49,0,0,50,0,2
7,Expert_7,48,1,0,49,1,0,50,0,0
8,Expert_8,48,1,1,49,1,0,50,2,1
9,Expert_9,48,1,0,49,0,0,50,1,0
